In [2]:
#Import bibliotek
pip install dash

import psycopg2
import pandas as pd
from sqlalchemy import create_engine
import dash
from dash.dependencies import Input, Output  # Dodano import Output
import plotly.express as px
import plotly.graph_objects as go
import dash_core_components as dcc
import dash_html_components as html
import dash_table

#Połączenie z bazą danych
username = 'nick'
password = 'haslo'
host = 'localhost'
database = 'airlines'
port = port

con = psycopg2.connect(
    dbname=database,
    user=username,
    password=password,
    host=host,
    port=port
)
cursor = con.cursor()



#Stworzenie tabeli dla `TOP 10 routes`

#Zdefiniowane zapytania SQL
query = "SELECT * FROM reporting.top_reliability_roads"

#Wczytanie danych z widoku do ramki danych
top_routes_df = pd.read_sql_query(query, con)

#Aktualizacja kolumny "reliability" na format procentowy
top_routes_df["reliability"] = top_routes_df["reliability"].apply(lambda x: f"{x:.2f}%")
top_routes_df

#Filtracja tylko istotnych kolumn
top_routes_df= top_routes_df[['origin_airport_name', 'dest_airport_name', 'year', 'reliability', 'nb']]

#Zmień nazwy kolumn
top_routes_df.rename(columns={'origin_airport_name': 'Origin', 'dest_airport_name': 'Destination','year': 'Year', 'reliability': 'Reliability', 'nb': 'Rank'}, inplace=True)
top_routes_df = top_routes_df.head(10)



#Porównanie rok do roku 2019 vs. 2020

#Zdefinie zapytania SQL
query2 = "SELECT * FROM reporting.year_to_year_comparision"

#Wczytanie danych z widoku do ramki danych
yoy_comparision_df = pd.read_sql_query(query2, con)

#Tabela przestawna
yoy_comparision_to_plot_df = yoy_comparision_df.pivot(index='month', columns='year', values='reliability')

#Wykres słupkowy
yoy_comparision_fig = px.bar(yoy_comparision_to_plot_df, x=yoy_comparision_to_plot_df.index, y=[2019, 2020], title="Reliability comparison (2019 vs. 2020)", color_discrete_sequence=['blue', 'green'])

yoy_comparision_fig.update_layout(
     title={
        'text': "Reliability comparison (2019 vs. 2020)",
        'x': 0.5,  # Ustawienie tytułu na środku
        'y': 0.95,  # Ustawienie tytułu na górze wykresu
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 16, 'family': 'Arial'}
    },
    xaxis_title="Month",
    yaxis_title="Reliability",
    legend_title="Year",
    barmode='group'
)

yoy_comparision_fig.update_xaxes(type='category', tickmode='array', tickvals=list(range(1, 13)), ticktext=['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'])

#Aplikacja Dash
app = dash.Dash(__name__)

#Definicja układu aplikacji
app.layout = html.Div([
    dcc.Graph(figure=yoy_comparision_fig)
])

#Uruchomienie aplikacji
if __name__ == '__main__':
    app.run_server(debug=True, port=8050)
    
    

#Porównanie dzień tygodnia do dnia tygodnia 2019 vs. 2020

#Zdefinie zapytania SQL
query3 = "SELECT * FROM reporting.day_to_day_comparision"

#Wczytanie danych z widoku do ramki danych
day_to_day_comparision_df = pd.read_sql_query(query3, con)

#Tabela przestawna
day_to_day_comparision_to_plot_df = day_to_day_comparision_df.pivot(index='day_of_week', columns='year', values='flights_amount')
day_to_day_comparision_to_plot_df

#Wykres słupkowy
day_to_day_comparision_fig = px.bar(day_to_day_comparision_to_plot_df, x=day_to_day_comparision_to_plot_df.index, y=[2019, 2020], title="Flight amount comparison (2019 vs. 2020)", color_discrete_sequence=['blue', 'green'])

day_to_day_comparision_fig.update_layout(
    title={
        'text': "Flight amount comparison (2019 vs. 2020)",
        'x': 0.5,  # Ustawienie tytułu na środku
        'y': 0.95,  # Ustawienie tytułu na górze wykresu
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 16, 'family': 'Arial'}
    },
    xaxis_title="Day of week",
    yaxis_title="Flights amount",
    legend_title="Year",
    barmode='group'
        )

day_to_day_comparision_fig.update_xaxes(type='category', tickmode='array', tickvals=list(range(1, 8)), ticktext=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])

#Tworzymy aplikację Dash
app_2 = dash.Dash(__name__)

#Definiujemy układ aplikacji
app_2.layout = html.Div([
    dcc.Graph(figure=day_to_day_comparision_fig)
])

#Uruchamiamy aplikację
if __name__ == '__main__':
    app_2.run_server(debug=True, port=8051)
    
    

#Stworzenie szeregu czasowego

#Zdefinie zapytania SQL
query4 = "SELECT * FROM reporting.day_by_day_reliability"

# Wczytanie danych z widoku do ramki danych
day_by_day_reliability_df = pd.read_sql_query(query4, con)

day_by_day_reliability_df.head()

#Wykres liniowy

#Konwersja kolumny 'date' na typ datetime
day_by_day_reliability_df['date'] = pd.to_datetime(day_by_day_reliability_df['date'])

#Filtracja danych dla roku 2019 i 2020
df_2019 = day_by_day_reliability_df[day_by_day_reliability_df['date'].dt.year == 2019]
df_2020 = day_by_day_reliability_df[day_by_day_reliability_df['date'].dt.year == 2020]

#Tworzenie wykresu
day_by_day_reliability_fig = go.Figure()

#Dodanie linii dla roku 2019
day_by_day_reliability_fig.add_trace(go.Scatter(x=df_2019['date'], y=df_2019['reliability'], mode='lines', name='2019', line=dict(color='blue')))

#Dodanie linii dla roku 2020
day_by_day_reliability_fig.add_trace(go.Scatter(x=df_2020['date'], y=df_2020['reliability'], mode='lines', name='2020', line=dict(color='green')))

#Konfiguracja osi x
day_by_day_reliability_fig.update_xaxes(
    tickformat="%d-%m",
    tickmode="linear",
    dtick="M1"
)

#Konfiguracja osi x i y oraz tytułu wykresu

day_by_day_reliability_fig.update_layout(
    title={
        'text': "Reliability over Time",
        'x': 0.5,  # Ustawienie tytułu na środku
        'y': 0.95,  # Ustawienie tytułu na górze wykresu
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 16, 'family': 'Arial'}
    },
    xaxis_title="Date",
    yaxis_title="Reliability",
    legend_title="Year",
    barmode='group'
)
        
# Tworzymy aplikację Dash
app_3 = dash.Dash(__name__)

# Definiujemy układ aplikacji
app_3.layout = html.Div([
    dcc.Graph(figure=day_by_day_reliability_fig)
])

# Uruchamiamy aplikację
if __name__ == '__main__':
    app_3.run_server(debug=True, port=8052)
    
    

#Stworzenie layoutów

#Przygotowanie komponentu Dash DataTable
top_routes_table = dash_table.DataTable(
    id='top-routes-table',
    columns=[{"name": i, "id": i} for i in top_routes_df.columns],
    data=top_routes_df.to_dict('records'),
    style_table={'overflowX': 'auto'},  # Dodatkowy styl dla tabeli
)

#Nagłówek
top_routes_page_title = html.H3(
    children='TOP 10 reliability routes in 2019 and 2020',
    style={'font-family': 'verdana', 'color': '#4444'}
)

#Układ strony dla widoku TOP 10 tras
top_routes_layout = html.Div([
    top_routes_page_title,
    top_routes_table
])



#Opakowanie yoy_comparision_fig w komponent Dash
yoy_comparision_component = dcc.Graph(figure=yoy_comparision_fig)

# Opakowanie day_to_day_comparision_fig w komponent Dash
day_to_day_comparision_component = dcc.Graph(figure=day_to_day_comparision_fig)

#Nagłówki dla wykresów
yoy_title = html.H3("Month over month comparison", style={'font-family': 'verdana', 'color': '#4444'})
day_to_day_title = html.H3("Day to day comparison", style={'font-family': 'verdana', 'color': '#4444'})

#Opisy dla wykresów
yoy_description = html.P("Comparison of reliability between 2019 and 2020", style={'font-family': 'verdana', 'color': '#6666'})
day_to_day_description = html.P("Comparison of flight amount between 2019 and 2020", style={'font-family': 'verdana', 'color': '#6666'})

# Utworzenie layoutu
comparision_layout = html.Div([
    yoy_title,
    yoy_description,
    yoy_comparision_component,  # Dodanie wykresu Year-over-year Comparison
    html.Br(),  # Dodanie odstępu
    day_to_day_title,
    day_to_day_description,
    day_to_day_comparision_component  # Dodanie wykresu Day-to-Day Comparison
])



#Nagłówek dla wykresu
day_by_day_title = html.H3("Day over day comparison", style={'font-family': 'verdana', 'color': '#4444'})

#Opis dla wykresu
day_by_day_description = html.P("Comparison of reliability between 2019 and 2020", style={'font-family': 'verdana', 'color': '#6666'})

day_by_day_layout = html.Div([
    day_by_day_title,
    day_by_day_description,
    dcc.Graph(figure=day_by_day_reliability_fig)
])

# Tworzenie aplikacji Dash
app_4 = dash.Dash(__name__)



#Layout aplikacji
app_4.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Nav([
        html.Ul([
            html.Li(html.A('TOP report', href='/')),
            html.Li(html.A('Comparison', href='/comparision_reporting')),
            html.Li(html.A('Day by day reporting', href='/day_by_day_reporting')),
        ])
    ]),
    html.Div(id='page-content')
])



#Stworzenie procedury odpowiadającej za nawigację pomiędzy poszczególnymi stronami
@app_4.callback(
    Output('page-content', 'children'),
    [Input('url', 'pathname')]
)
def display_page(pathname):
    if pathname == '/comparision_reporting':
        return comparision_layout
    elif pathname == '/day_by_day_reporting':
        return day_by_day_layout
    else:
        return top_routes_layout
    

# Uruchomienie aplikacji
if __name__ == '__main__':
    app_4.run_server(debug=True, port=8053)

  Obtaining dependency information for dash from https://files.pythonhosted.org/packages/b0/68/781d0026a100106b64e4501c76621dfcd0d3c29a546094fcffaa73037a74/dash-2.16.1-py3-none-any.whl.metadata
  Obtaining dependency information for dash-html-components==2.0.0 from https://files.pythonhosted.org/packages/75/65/1b16b853844ef59b2742a7de74a598f376ac0ab581f0dcc34db294e5c90e/dash_html_components-2.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for dash-core-components==2.0.0 from https://files.pythonhosted.org/packages/00/9e/a29f726e84e531a36d56cff187e61d8c96d2cc253c5bcef9a7695acb7e6a/dash_core_components-2.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for dash-table==5.0.0 from https://files.pythonhosted.org/packages/da/ce/43f77dc8e7bbad02a9f88d07bf794eaf68359df756a28bb9f2f78e255bb1/dash_table-5.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for retrying from https://files.pythonhosted.org/packages/8f/04/9e36f28be4c0532c0e9207ff9dc01fb